In [4]:
import datetime
from packaging import version
from collections import Counter
import numpy as np
import pandas as pd
import time
import os
import re
import string

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import nltk
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as k

#### Visualization Functions

In [5]:
def print_validation_report(test_labels, predictions):
    print("Classification Report")
    print(classification_report(test_labels, predictions))
    print('Accuracy Score: {}'.format(accuracy_score(test_labels, predictions)))
    print('Root Mean Square Error: {}'.format(np.sqrt(MSE(test_labels, predictions))))

def plot_confusion_matrix(y_true, y_pred):
    mtx = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.heatmap(mtx, annot=True, fmt='d', linewidths=.75,  cbar=False, ax=ax,cmap='Blues',linecolor='white')
    #  square=True,
    plt.ylabel('true label')
    plt.xlabel('predicted label')

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

#### Import Data

In [18]:
# register  ag_news_subset so that tfds.load doesn't generate a checksum (mismatch) error
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=ag_news_subset

dataset,info=\
tfds.load('ag_news_subset', with_info=True,  split=['train[:95%]','train[95%:]', 'test'],batch_size = 32
          , as_supervised=True)

train_ds, val_ds, test_ds = dataset
text_only_train_ds = train_ds.map(lambda x, y: x)

/opt/anaconda3/bin/python: Error while finding module specification for 'tensorflow_datasets.scripts.download_and_prepare' (ModuleNotFoundError: No module named 'tensorflow_datasets')


#### Data Preprocessing

In [23]:
nltk.download('stopwords',quiet=True)
STOPWORDS = stopwords.words("english")

In [24]:
def custom_stopwords(input_text):
    lowercase = tf.strings.lower(input_text)
    stripped_punct = tf.strings.regex_replace(lowercase
                                  ,'[%s]' % re.escape(string.punctuation)
                                  ,'')
    return tf.strings.regex_replace(stripped_punct, r'\b(' + r'|'.join(STOPWORDS) + r')\b\s*',"")

In [21]:
max_length = 96
max_tokens = 1000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
    standardize=custom_stopwords
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

#### Experiment 1 - 1D CNN

In [25]:
# output_sequence_length=96 # CONSTANT
dropout = 0.2 # CONSTANT
kernel_size = 3 # VARIABLE
filters = 32 # CONSTANT
input_dim = 1000 # CONSTANT, embedded one-hot encoding depth
units = 256 # CONSTANT

In [26]:
k.clear_session()

inputs = tf.keras.Input(shape=(None,), dtype="int64")

embedded = tf.one_hot(inputs, depth=input_dim)

x = layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(embedded)
x = layers.Dropout(dropout)(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(units, activation='relu')(x)

outputs = layers.Dense(4, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("LSTM.h5",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
]

history=model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)

model = keras.models.load_model("LSTM.h5")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

y_test = np.concatenate([y for x, y in int_test_ds], axis=0)

pred_classes = np.argmax(model.predict(int_test_ds), axis=-1)

Epoch 1/200
3563/3563 [==============================] - 32s 9ms/step - loss: 0.5298 - accuracy: 0.8086 - val_loss: 0.4226 - val_accuracy: 0.8528
Epoch 2/200
3563/3563 [==============================] - 32s 9ms/step - loss: 0.4304 - accuracy: 0.8478 - val_loss: 0.4163 - val_accuracy: 0.8587
Epoch 3/200
3563/3563 [==============================] - 33s 9ms/step - loss: 0.4228 - accuracy: 0.8515 - val_loss: 0.4167 - val_accuracy: 0.8600
Epoch 4/200
3563/3563 [==============================] - 32s 9ms/step - loss: 0.4208 - accuracy: 0.8535 - val_loss: 0.4185 - val_accuracy: 0.8592
Epoch 5/200
3563/3563 [==============================] - 31s 9ms/step - loss: 0.4202 - accuracy: 0.8541 - val_loss: 0.4259 - val_accuracy: 0.8557
Epoch 6/200
238/238 [==============================] - 1s 4ms/step - loss: 0.4287 - accuracy: 0.8488
Test acc: 0.849
238/238 [==============================] - 1s 4ms/step


#### Experiment 2

#### Experiment 3

#### Experiment 4

#### Experiment 5

#### Experiment 6

#### Experiment 7

#### Experiment 8

#### Experiment 9

#### Experiment 10

k.clear_session()
inputs = tf.keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens
                            ,output_dim=256
                            ,mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("LSTM.h5",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
]
history=model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)
model = keras.models.load_model("LSTM.h5")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

y_test = np.concatenate([y for x, y in int_test_ds], axis=0)
pred_classes = np.argmax(model.predict(int_test_ds), axis=-1)